In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
import pandas as pd
import numpy as np
import h5py
import joblib

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import FunctionTransformer, MinMaxScaler, StandardScaler
from sklearn.pipeline import make_pipeline

# Data Processing

In [2]:
def load_data(filename):
    """
    이미지를 불러옵니다.
    Parameters:
        filename: str
            h5 파일에서 데이터를 불러옵니다.
    Returns:
        np.ndarray, pd.DataFrame, np.ndarray, 
        train 이미지, train spot 정보, test 이미지, test spot 정보
    """
    images, images_test = list(), list()
    spots, spots_test = list(), list()
    with h5py.File(filename, "r") as h5file:
        train_images = h5file["images/Train"]
        train_spots = h5file["spots/Train"]
    
        num_train_slides = len(train_images)
        # Train 이미지를 불러옵니다.
        # 하나의 텐서로 만들기 위해 이미지의 크기를 2000x2000으로 균일하게 만듭니다.
        for i, slide_name in enumerate(train_images.keys()):
            image = np.array(train_images[slide_name])
            p1 = 2000 - image.shape[0]
            p2 = 2000 - image.shape[1]
            images.append(
                np.pad(image, [(0, p1), (0, p2), (0, 0)], 'edge')
            )
            spots.append(pd.DataFrame(np.array(train_spots[slide_name])).assign(slide = i))
        # Test 이미지를 불러옵니다.
        test_images = h5file["images/Test"]
        test_spots = h5file["spots/Test"]
        sample = 'S_7'
        image = np.array(test_images[sample])
        p1 = 2000 - image.shape[0]
        p2 = 2000 - image.shape[1]
        images_test.append(np.pad(image, [(0, p1), (0, p2), (0, 0)], 'edge'))
        spots_test.append(pd.DataFrame(np.array(test_spots[sample])).assign(slide = 0))
    # EfficientNet의 형식으로 바꿉니다.
    with tf.device('/CPU:0'):
        images = tf.constant(tf.keras.applications.efficientnet.preprocess_input(images))
    df_spots = pd.concat(spots)
    with tf.device('/CPU:0'):
        images_test = tf.constant(tf.keras.applications.efficientnet.preprocess_input(images_test))
    df_spots_test = pd.concat(spots_test)
    return images, df_spots, images_test, df_spots_test

def make_img_proc_info(df, img_with, img_height):
    """
    
    """
    return df.assign(
        left = lambda x: x['x'] - img_width // 2,
        right = lambda x: x['x'] + img_width // 2,
        top = lambda x: x['y'] - img_height // 2,
        bottom = lambda x: x['y'] + img_height // 2,
        lpad = lambda x: -(x['left'].where(x['left'] < 0, 0)),
        rpad = lambda x: -(2000 - x['right']).where(x['right'] > 2000, 0),
        tpad = lambda x: -(x['top'].where(x['top'] < 0, 0)),
        bpad = lambda x: -(2000 - x['bottom']).where(x['bottom'] > 2000, 0)
    ).assign(
        left = lambda x: x['left'].clip(0, 2000),
        right = lambda x: x['right'].clip(0, 2000),
        top = lambda x: x['top'].clip(0, 2000),
        bottom = lambda x: x['bottom'].clip(0, 2000),
    )

def create_tf_ds(df):
    if (pd.Series(targets).isin(df.columns)).all():
        return tf.data.Dataset.from_tensor_slices(
            ({
                i: df[i] for i in ['left', 'right', 'top', 'bottom', 'slide', 'lpad', 'rpad', 'tpad', 'bpad']
            }, df[targets])
        )
    else:
        return tf.data.Dataset.from_tensor_slices({
            i: df[i] for i in ['left', 'right', 'top', 'bottom', 'slide', 'lpad', 'rpad', 'tpad', 'bpad']
        })

def proc_images(X, images):
    return tf.pad(
        images[X['slide'], X['left']:X['right'], X['top']:X['bottom'], :], 
        paddings = [(X['lpad'], X['rpad']), (X['tpad'], X['bpad']), (0, 0)],
        constant_values=1
    )

augmentation_layers = [
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
]

def data_augmentation(x):
    for layer in augmentation_layers:
        x = layer(x)
    return x

images, df_spots, images_test, df_spots_test = load_data("data/elucidata_ai_challenge_data.h5")
targets = [i for i in df_spots.columns if i.startswith('C')]

target_proc = make_pipeline(FunctionTransformer(np.log, np.exp),  StandardScaler())
target_proc.fit(df_spots[targets])
df_spots[targets] = target_proc.transform(df_spots[targets])

In [3]:
img_width = 224
img_height = 224

df_spots = make_img_proc_info(df_spots, img_width, img_height)
df_spots_test = make_img_proc_info(df_spots_test, img_width, img_height)

In [4]:
df_spots['slide'].unique()

array([0, 1, 2, 3, 4, 5])

In [5]:
from tqdm.notebook import tqdm
class TqdmEpochProgress(tf.keras.callbacks.Callback):
    def __init__(self, epochs):
        super().__init__()
        self.epochs = epochs
        self.progress_bar = None

    def on_train_begin(self, logs=None):
        self.progress_bar = tqdm(total=self.epochs, desc="Epochs")

    def on_epoch_end(self, epoch, logs=None):
        log_str = f"loss: {logs.get('loss'):.4f}"
        if 'val_loss' in logs:
            log_str += f", val_loss: {logs.get('val_loss'):.4f}"
        self.progress_bar.set_postfix_str(log_str)
        self.progress_bar.update(1)

    def on_train_end(self, logs=None):
        self.progress_bar.close()

# Validation

In [39]:
batch_size = 32
ds_cv_train = create_tf_ds(
    df_spots.loc[df_spots['slide'] != 5].pipe(
        lambda x: pd.concat([
            x, x.sample(n = batch_size - (len(x) % batch_size))
        ])
    )
).shuffle(5000).map(
    lambda X, Y: (proc_images(X, images), Y)
).map(
    lambda X, Y: (data_augmentation(X), Y)
).batch(batch_size).prefetch(tf.data.AUTOTUNE).cache()

ds_valid = create_tf_ds(df_spots.loc[df_spots['slide'] == 5]).map(
    lambda X, Y: (proc_images(X, images), Y)
).batch(batch_size).prefetch(tf.data.AUTOTUNE).cache()

In [55]:
from scipy.stats import spearmanr
from sklearn.metrics import mean_squared_error

input_shape = (img_width, img_height, 3)
enet = tf.keras.applications.EfficientNetB0(
    include_top = False, 
    weights = 'imagenet',
    input_shape = input_shape,
    pooling = 'avg'
)
inputs = tf.keras.Input(shape = input_shape)
x = enet(inputs, training = False)
x = tf.keras.layers.Dropout(0.2)(x)
d1 = tf.keras.layers.Dense(64, activation = 'relu', kernel_initializer = 'HeUniform')
x = d1(x)
d2 = tf.keras.layers.Dense(len(targets))
outputs = d2(x)
m = tf.keras.models.Model(inputs, outputs)

lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=3e-6,
    decay_steps=5000,
    alpha=0.1
)

m.compile(
    loss = tf.keras.losses.MeanSquaredError(),
    optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule),
    metrics = [tf.keras.metrics.MeanSquaredError()]
)
df_true = pd.DataFrame(
    target_proc.inverse_transform(
        df_spots.loc[df_spots['slide'] == 5, targets]
    ), index = df_spots[df_spots['slide'] == 5].index, columns = targets
)
for i in range(30):
    hist = m.fit(ds_cv_train, epochs = 1)
    df_prd = pd.DataFrame(
        target_proc.inverse_transform(m.predict(ds_valid)), index = df_spots[df_spots['slide'] == 5].index, columns = targets
    )
    print(
        df_true.apply(lambda x: spearmanr(x, df_prd.loc[x.name])[0], axis=1).mean(),
        mean_squared_error(df_true.stack(), df_prd.stack())
    )

251/251 ━━━━━━━━━━━━━━━━━━━━ 48s 76ms/step - loss: 1.1092 - mean_squared_error: 1.1092
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 342ms/step
0.518999081294782 0.33009170559704354
251/251 ━━━━━━━━━━━━━━━━━━━━ 19s 76ms/step - loss: 1.0521 - mean_squared_error: 1.0521
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
0.5207991733278677 0.3290757018812078
251/251 ━━━━━━━━━━━━━━━━━━━━ 19s 75ms/step - loss: 1.0219 - mean_squared_error: 1.0219
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
0.5601591856254697 0.32236360870606423
251/251 ━━━━━━━━━━━━━━━━━━━━ 21s 84ms/step - loss: 0.9986 - mean_squared_error: 0.9986
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
0.5699605451936872 0.3200590939559671
251/251 ━━━━━━━━━━━━━━━━━━━━ 20s 79ms/step - loss: 0.9810 - mean_squared_error: 0.9810
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
0.5653822504611601 0.3187721206633479
251/251 ━━━━━━━━━━━━━━━━━━━━ 19s 76ms/step - loss: 0.9682 - mean_squared_error: 0.9682
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
0.5683396529343445 0.3181607028137847
251/251 

# Fine Tuning


In [58]:
inputs = tf.keras.Input(shape = input_shape)
x = enet(inputs, training = True)
x = tf.keras.layers.Dropout(0.2)(x)
x = d1(x)
outputs = d2(x)
m = tf.keras.models.Model(inputs, outputs)
m.compile(
    optimizer=tf.keras.optimizers.Adam(3e-6),  # Low learning rate
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.MeanSquaredError()],
)
for i in range(10):
    hist = m.fit(ds_cv_train, epochs=1)
    df_prd = pd.DataFrame(
        target_proc.inverse_transform(m.predict(ds_valid)), index = df_spots[df_spots['slide'] == 5].index, columns = targets
    )
    print(
        df_true.apply(lambda x: spearmanr(x, df_prd.loc[x.name])[0], axis=1).mean(),
        mean_squared_error(df_true.stack(), df_prd.stack())
    )

251/251 ━━━━━━━━━━━━━━━━━━━━ 50s 78ms/step - loss: 0.8075 - mean_squared_error: 0.8075
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 301ms/step
0.5723167315706771 0.310563293689337
251/251 ━━━━━━━━━━━━━━━━━━━━ 19s 76ms/step - loss: 0.8002 - mean_squared_error: 0.8002
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
0.5730656896905104 0.31008050635853646
251/251 ━━━━━━━━━━━━━━━━━━━━ 21s 84ms/step - loss: 0.7902 - mean_squared_error: 0.7902
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
0.5714456514313042 0.3101638124620468
251/251 ━━━━━━━━━━━━━━━━━━━━ 19s 76ms/step - loss: 0.7807 - mean_squared_error: 0.7807
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
0.5698879551820728 0.3100369421991883
251/251 ━━━━━━━━━━━━━━━━━━━━ 19s 76ms/step - loss: 0.7722 - mean_squared_error: 0.7722
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
0.5694387511102001 0.30977486910956953
251/251 ━━━━━━━━━━━━━━━━━━━━ 21s 83ms/step - loss: 0.7638 - mean_squared_error: 0.7638
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
0.5684754389560703 0.31028821340196205
251/251

# Train

In [6]:
batch_size = 32
ds_train = create_tf_ds(
    df_spots.pipe(
        lambda x: pd.concat([
            x, x.sample(n = batch_size - (len(x) % batch_size))
        ])
    )
).shuffle(5000).map(
    lambda X, Y: (proc_images(X, images), Y)
).map(
    lambda X, Y: (data_augmentation(X), Y)
).batch(batch_size).prefetch(tf.data.AUTOTUNE).cache()

input_shape = (img_width, img_height, 3)
enet = tf.keras.applications.EfficientNetB0(
    include_top = False, 
    weights = 'imagenet',
    input_shape = input_shape,
    pooling = 'avg'
)
inputs = tf.keras.Input(shape = input_shape)
x = enet(inputs, training = False)
x = tf.keras.layers.Dropout(0.2)(x)
d1 = tf.keras.layers.Dense(64, activation = 'relu', kernel_initializer = 'HeUniform')
x = d1(x)
d2 = tf.keras.layers.Dense(len(targets))
outputs = d2(x)
m = tf.keras.models.Model(inputs, outputs)

lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=3e-6,
    decay_steps=5000,
    alpha=0.1
)

m.compile(
    loss = tf.keras.losses.MeanSquaredError(),
    optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule),
    metrics = [tf.keras.metrics.MeanSquaredError()]
)
hist = m.fit(ds_train, epochs = 30)

Epoch 1/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 90s 158ms/step - loss: 1.1102 - mean_squared_error: 1.1102
Epoch 2/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 22s 83ms/step - loss: 1.0492 - mean_squared_error: 1.0492
Epoch 3/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 19s 74ms/step - loss: 1.0132 - mean_squared_error: 1.0132
Epoch 4/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 22s 75ms/step - loss: 0.9911 - mean_squared_error: 0.9911
Epoch 5/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 19s 74ms/step - loss: 0.9722 - mean_squared_error: 0.9722
Epoch 6/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 22s 84ms/step - loss: 0.9585 - mean_squared_error: 0.9585
Epoch 7/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 20s 75ms/step - loss: 0.9460 - mean_squared_error: 0.9460
Epoch 8/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 19s 74ms/step - loss: 0.9372 - mean_squared_error: 0.9372
Epoch 9/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 19s 74ms/step - loss: 0.9273 - mean_squared_error: 0.9273
Epoch 10/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 22s 84ms/step - loss: 0.9223 - mean_squared_error: 0.9223
Epoch 11/30
261/26

In [7]:
inputs = tf.keras.Input(shape = input_shape)
x = enet(inputs, training = True)
x = tf.keras.layers.Dropout(0.2)(x)
x = d1(x)
outputs = d2(x)
m = tf.keras.models.Model(inputs, outputs)
m.compile(
    optimizer=tf.keras.optimizers.Adam(1e-6),  # Low learning rate
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.MeanSquaredError()],
)
hist = m.fit(ds_train, epochs=10)

Epoch 1/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 53s 75ms/step - loss: 0.8789 - mean_squared_error: 0.8789
Epoch 2/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 22s 84ms/step - loss: 0.8748 - mean_squared_error: 0.8748
Epoch 3/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 20s 76ms/step - loss: 0.8705 - mean_squared_error: 0.8705
Epoch 4/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 19s 73ms/step - loss: 0.8666 - mean_squared_error: 0.8666
Epoch 5/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 19s 74ms/step - loss: 0.8645 - mean_squared_error: 0.8645
Epoch 6/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 22s 83ms/step - loss: 0.8619 - mean_squared_error: 0.8619
Epoch 7/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 19s 73ms/step - loss: 0.8578 - mean_squared_error: 0.8578
Epoch 8/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 19s 73ms/step - loss: 0.8542 - mean_squared_error: 0.8542
Epoch 9/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 25s 95ms/step - loss: 0.8518 - mean_squared_error: 0.8518
Epoch 10/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 17s 83ms/step - loss: 0.8491 - mean_squared_error: 0.8491


In [8]:
joblib.dump(m.get_weights(), 'model/eff_b0_2.joblib')

['model/eff_b0_2.joblib']

In [9]:
joblib.dump(target_proc, 'model/target_proc_2.joblib') 

['model/target_proc_2.joblib']

In [10]:
ds_test = create_tf_ds(df_spots_test)

df_submission = pd.DataFrame(
    target_proc.inverse_transform(
        m.predict(
            ds_test.map(lambda X: proc_images(X, images_test)).batch(32)
        )
    ), columns = targets
).reset_index().rename(columns = {'index': 'ID'})

66/66 ━━━━━━━━━━━━━━━━━━━━ 12s 127ms/step


In [11]:
df_submission

,ID,C1,C2,C3,C4,C5,C6,C7,C8,C9,...,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35
0,0,0.139854,0.064018,0.076188,0.044425,0.087721,0.005078,0.012797,0.000573,0.003890,...,0.001139,0.001088,0.000957,0.002257,0.000115,0.052555,0.024842,0.000521,0.004712,0.020507
1,1,0.051364,0.077110,0.129357,0.021935,0.007879,0.002041,0.007267,0.001011,0.001386,...,0.004315,0.000774,0.000674,0.000858,0.000226,0.041766,0.025205,0.001300,0.005841,0.038908
2,2,0.052973,0.074979,0.084442,0.027101,0.060929,0.010806,0.011961,0.001323,0.002091,...,0.002159,0.000674,0.000901,0.001306,0.000196,0.048293,0.026916,0.001000,0.002311,0.028778
3,3,0.084694,0.088639,0.127371,0.023594,0.041324,0.003887,0.011799,0.000987,0.012646,...,0.007179,0.000669,0.000652,0.003581,0.000086,0.032791,0.031920,0.000718,0.003610,0.030696
4,4,0.235286,0.093400,0.089184,0.010078,0.019840,0.000833,0.004649,0.001079,0.000704,...,0.000595,0.000231,0.001385,0.000641,0.000959,0.032764,0.038569,0.002261,0.003929,0.028193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2083,2083,0.077080,0.094566,0.290037,0.054553,0.165372,0.002030,0.021844,0.000932,0.001730,...,0.001904,0.005125,0.000803,0.002163,0.000060,0.055566,0.075000,0.001859,0.001565,0.037228
2084,2084,0.156940,0.089073,0.160257,0.069841,0.197851,0.004648,0.016571,0.000416,0.003962,...,0.001765,0.004047,0.001326,0.003371,0.000028,0.052311,0.045426,0.001010,0.003696,0.018575
2085,2085,0.234652,0.053667,0.182117,0.091130,0.040251,0.002700,0.020946,0.001584,0.003226,...,0.003690,0.001561,0.000360,0.002399,0.000065,0.056341,0.042913,0.000556,0.005951,0.027432
2086,2086,0.095310,0.047352,0.179007,0.018158,0.042264,0.005965,0.013608,0.000725,0.003086,...,0.001524,0.001644,0.000918,0.003246,0.000061,0.049831,0.041827,0.001535,0.002553,0.026203
